# Trexquant Interview Project (The Hangman Game)

* Copyright Trexquant Investment LP. All Rights Reserved.
* Redistribution of this question without written consent from Trexquant is prohibited

## Instruction:
For this coding test, your mission is to write an algorithm that plays the game of Hangman through our API server.

When a user plays Hangman, the server first selects a secret word at random from a list. The server then returns a row of underscores (space separated)—one for each letter in the secret word—and asks the user to guess a letter. If the user guesses a letter that is in the word, the word is redisplayed with all instances of that letter shown in the correct positions, along with any letters correctly guessed on previous turns. If the letter does not appear in the word, the user is charged with an incorrect guess. The user keeps guessing letters until either (1) the user has correctly guessed all the letters in the word
or (2) the user has made six incorrect guesses.

You are required to write a "guess" function that takes current word (with underscores) as input and returns a guess letter. You will use the API codes below to play 1,000 Hangman games. You have the opportunity to practice before you want to start recording your game results.

Your algorithm is permitted to use a training set of approximately 250,000 dictionary words. Your algorithm will be tested on an entirely disjoint set of 250,000 dictionary words. Please note that this means the words that you will ultimately be tested on do NOT appear in the dictionary that you are given. You are not permitted to use any dictionary other than the training dictionary we provided. This requirement will be strictly enforced by code review.

You are provided with a basic, working algorithm. This algorithm will match the provided masked string (e.g. a _ _ l e) to all possible words in the dictionary, tabulate the frequency of letters appearing in these possible words, and then guess the letter with the highest frequency of appearence that has not already been guessed. If there are no remaining words that match then it will default back to the character frequency distribution of the entire dictionary.

This benchmark strategy is successful approximately 18% of the time. Your task is to design an algorithm that significantly outperforms this benchmark.

In [3]:
import json
import requests
import random
import string
import secrets
import time
import re
import collections
import numpy as np
import pandas as pd


try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [4]:
from tokenize import String
class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.hangman_url = self.determine_hangman_url()
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        self.guessed_letters = []

        full_dictionary_location = "words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()

        self.current_dictionary = []
        # declare frequency table for different patterns
        self.freq_table1 = np.array([]) # pattern of lenght 1
        self.freq_table2 = np.array([]) # pattern of lenght 2
        self.freq_table3 = np.array([]) # pattern of lenght 3
        self.freq_table4 = np.array([]) # pattern of lenght 4
        self.freq_table5 = np.array([]) # pattern of lenght 5
        self.build_freq_tables()
        # declare a array to track which letters have been guessed
        self.not_possible = np.zeros(28, dtype = np.int32)


    @staticmethod
    def determine_hangman_url():
        links = ['https://trexsim.com', 'https://sg.trexsim.com']

        data = {link: 0 for link in links}

        for link in links:

            requests.get(link)

            for i in range(10):
                s = time.time()
                requests.get(link)
                data[link] = time.time() - s

        link = sorted(data.items(), key=lambda x: x[1])[0][0]
        link += '/trexsim/hangman'
        return link

    def guess(self, word): # word input example: "_ p p _ e "
        word = word.replace(' ', '')
        word = "{" + word + "|"
        score = np.zeros(28)
        coeff = np.array([1,1,4,10, 20])
        for i in range(5):
            score = score + self.model(word, i+1)*coeff[i]
        guess_letter = chr(score.argmax() + ord('a'))
        self.not_possible[score.argmax()] = 1
        return guess_letter

    ##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################

    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary

    def build_freq_tables(self):
        self.freq_table1 = np.zeros(28)
        self.freq_table2 = np.zeros((28, 28))
        self.freq_table3 = np.zeros((28, 28, 28))
        self.freq_table4 = np.zeros((28, 28, 28, 28))
        self.freq_table5 = np.zeros((28, 28, 28, 28, 28))
        for word in self.full_dictionary:
            # padding for end recognition
            word = "{" + word + "|"
            
            # table1
            unique_letters = np.zeros(28)
            for ch in word:
                a = ord(ch) - ord('a')
                unique_letters[a] = 1
            for i in range(28):
                if unique_letters[i] == 1 :
                    self.freq_table1[i] = self.freq_table1[i] + 1

            # table 2
            i = 0
            while i + 1 < len(word):
                a = ord(word[i]) -ord('a')
                b = ord(word[i+1]) -ord('a')
                self.freq_table2[a, b] = self.freq_table2[a, b] + 1
                i = i + 1

            # table 3
            i = 0
            while i + 2 < len(word):
                a = ord(word[i]) -ord('a')
                b = ord(word[i+1]) -ord('a')
                c = ord(word[i+2]) -ord('a')
                self.freq_table3[a, b, c] = self.freq_table3[a, b, c] + 1
                i = i + 1

            # table 4
            i = 0
            while i + 3 < len(word):
                a = ord(word[i]) -ord('a')
                b = ord(word[i+1]) -ord('a')
                c = ord(word[i+2]) -ord('a')
                d = ord(word[i+3]) -ord('a')
                self.freq_table4[a, b, c, d] = self.freq_table4[a, b, c, d] + 1
                i = i + 1

            # table 5
            i = 0
            while i + 4 < len(word):
                a = ord(word[i]) -ord('a')
                b = ord(word[i+1]) -ord('a')
                c = ord(word[i+2]) -ord('a')
                d = ord(word[i+3]) -ord('a')
                e = ord(word[i+4]) -ord('a')
                self.freq_table5[a, b, c, d, e] = self.freq_table5[a, b, c, d, e] + 1
                i = i + 1

    def normalise_prob_array( self, prob):
        sum  = 0
        for i in range(28) :
            if self.not_possible[i] == 1:
                prob[i] = 0
            else :
                sum = sum + prob[i]
        if sum<1: # if the array is all zeros, we revert back to the base probabilities
            sum = 0
            for j in range(28):
                if self.not_possible[j] == 0:
                    prob[j] = self.freq_table1[j]
                    sum = sum + self.freq_table1[j]
            return prob/sum
        return prob/sum

    def Conditional_Probability(self, pattern):
        freq = np.zeros(28)
        n = len(pattern)
        for i in range(28):
            if self.not_possible[i] == 1:
                continue

            p = pattern.replace('_', chr(ord('a')+i))
            if n == 2:
                a = ord(p[0]) - ord('a')
                b = ord(p[1]) - ord('a')
                freq[i] = freq[i] + self.freq_table2[a, b]
            elif n == 3:
                a = ord(p[0]) - ord('a')
                b = ord(p[1]) - ord('a')
                c = ord(p[2]) - ord('a')
                freq[i] = freq[i] + self.freq_table3[a, b, c]

            elif n == 4:
                a = ord(p[0]) - ord('a')
                b = ord(p[1]) - ord('a')
                c = ord(p[2]) - ord('a')
                d = ord(p[3]) - ord('a')
                freq[i] = freq[i] + self.freq_table4[a, b, c, d]
            elif n == 5:
                a = ord(p[0]) - ord('a')
                b = ord(p[1]) - ord('a')
                c = ord(p[2]) - ord('a')
                d = ord(p[3]) - ord('a')
                e = ord(p[4]) - ord('a')
                freq[i] = freq[i] + self.freq_table5[a, b, c, d, e]
        
        result = self.normalise_prob_array(freq)
        
        return result

    def model(self , word, length):
        score = np.zeros(28)
        i = 0
        while i + length - 1 <  len(word):
            p =  word[i:(i+length)]

            if p.count('_') == 1:
                score = score + self.Conditional_Probability(p)
                
            i = i+1
        return score

    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary
        self.not_possible = np.zeros(28)
        self.not_possible[26] = 1
        self.not_possible[27] = 1

        response = self.request("/new_game", {"practice":practice})
        if response.get('status')=="approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')
            if verbose:
                print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, tries_remains, word))
            while tries_remains>0:
                # get guessed letter from user code
                guess_letter = self.guess(word)

                # append guessed letter to guessed letters field in hangman object
                self.guessed_letters.append(guess_letter)
                self.not_possible[ord(guess_letter) - ord('a')] = 1
                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))

                try:
                    res = self.request("/guess_letter", {"request":"guess_letter", "game_id":game_id, "letter":guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e

                if verbose:
                    print("Sever response: {0}".format(res))
                status = res.get('status')
                tries_remains = res.get('tries_remains')
                if status=="success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    return True
                elif status=="failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))
                    return False
                elif status=="ongoing":
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return status=="success"

    def my_status(self):
        return self.request("/my_status", {})

    def request(
            self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        # Add `access_token` to post_args or args if it has not already been
        # included.
        if self.access_token:
            # If post_args exists, we assume that args either does not exists
            # or it does not need `access_token`.
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        time.sleep(0.2)

        num_retry, time_sleep = 50, 2
        for it in range(num_retry):
            try:
                response = self.session.request(
                    method or "GET",
                    self.hangman_url + path,
                    timeout=self.timeout,
                    params=args,
                    data=post_args,
                    verify=False
                )
                break
            except requests.HTTPError as e:
                response = json.loads(e.read())
                raise HangmanAPIError(response)
            except requests.exceptions.SSLError as e:
                if it + 1 == num_retry:
                    raise
                time.sleep(time_sleep)

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result

class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)




# API Usage Examples

## To start a new game:
1. Make sure you have implemented your own "guess" method.
2. Use the access_token that we sent you to create your HangmanAPI object.
3. Start a game by calling "start_game" method.
4. If you wish to test your function without being recorded, set "practice" parameter to 1.
5. Note: You have a rate limit of 20 new games per minute. DO NOT start more than 20 new games within one minute.

In [ ]:
api = HangmanAPI(access_token="d93cc8b042d9712516bc9fbf70bc54", timeout=2000)

## Playing practice games:
You can use the command below to play up to 100,000 practice games.

In [99]:
for i in range(1000):
    api.start_game(practice=1,verbose=False)
    [total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
    practice_success_rate = total_practice_successes / total_practice_runs
    print('run %d practice games out of an allotted 100,000. practice success rate so far = %.3f' % (total_practice_runs, practice_success_rate))


run 1001 practice games out of an allotted 100,000. practice success rate so far = 0.564
run 1002 practice games out of an allotted 100,000. practice success rate so far = 0.565
run 1003 practice games out of an allotted 100,000. practice success rate so far = 0.565
run 1004 practice games out of an allotted 100,000. practice success rate so far = 0.566
run 1005 practice games out of an allotted 100,000. practice success rate so far = 0.565
run 1006 practice games out of an allotted 100,000. practice success rate so far = 0.565
run 1007 practice games out of an allotted 100,000. practice success rate so far = 0.564
run 1008 practice games out of an allotted 100,000. practice success rate so far = 0.563
run 1009 practice games out of an allotted 100,000. practice success rate so far = 0.564
run 1010 practice games out of an allotted 100,000. practice success rate so far = 0.563
run 1011 practice games out of an allotted 100,000. practice success rate so far = 0.564
run 1012 practice gam

## Playing recorded games:
Please finalize your code prior to running the cell below. Once this code executes once successfully your submission will be finalized. Our system will not allow you to rerun any additional games.

Please note that it is expected that after you successfully run this block of code that subsequent runs will result in the error message "Your account has been deactivated".

Once you've run this section of the code your submission is complete. Please send us your source code via email.

In [105]:
for i in range(1000):
    print('Playing ', i, ' th game')
    # Uncomment the following line to execute your final runs. Do not do this until you are satisfied with your submission
    api.start_game(practice=0,verbose=False)

    # DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
    time.sleep(0.5)

Playing  0  th game


HangmanAPIError: {'error': 'Your account has been deactivated!'}

## To check your game statistics
1. Simply use "my_status" method.
2. Returns your total number of games, and number of wins.

In [5]:
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
success_rate = total_recorded_successes/total_recorded_runs
print('overall success rate = %.3f' % success_rate)
print(total_recorded_runs)

overall success rate = 0.600
1000
